In [328]:
using JuMP, Gurobi, JSON, DataFrames
# JuMP v1.13.0
# Julia v1.8.5
# Gurobi v1.0.2
# JSON v0.21.4
# DataFrame v1.5.0

include("formatting.jl")
include("hueristic.jl")
include("import.jl")
include("model_relaxation.jl")
include("schedules.jl")

DATA_PATH = "../data/simple"

"../data/simple"

In [329]:
𝓓::Data = import_data(DATA_PATH, 15)

Data(NamedTuple{(:name, :min_capacity, :max_capacity, :initial_volume, :rates, :flows, :type), Tuple{String, Int64, Int64, Int64, Vector{Any}, Vector{Any}, String}}[(name = "silo1", min_capacity = 0, max_capacity = 1000, initial_volume = 900, rates = [], flows = ["machine1"], type = "storage"), (name = "silo3", min_capacity = 0, max_capacity = 600, initial_volume = 300, rates = [], flows = ["machine2"], type = "storage"), (name = "silo2", min_capacity = 0, max_capacity = 500, initial_volume = 200, rates = [], flows = ["machine1", "machine2"], type = "storage"), (name = "cleaner1", min_capacity = 0, max_capacity = 1, initial_volume = 1, rates = [-1, -1], flows = ["machine1", "machine2"], type = "renewable")], NamedTuple{(:name, :initial_state, :initial_time, :run_time, :cleaning_time, :min_off_time, :max_off_time, :rates, :flows, :cleaner_key), Tuple{String, String, Int64, Int64, Int64, Int64, Int64, Vector{Any}, Vector{Any}, String}}[(name = "machine2", initial_state = "on", initial_ti

In [330]:
itteration = 1
schedules_dict = generate_schedules_for_machines(𝓓, Dict(), itteration, true)
println(schedules_dict)

Dict{Any, Any}(("machine1", 1) => Any[state("on", 2), state("idle", 0), state("cleaning", 3), state("off", 2), state("on", 7), state("idle", 0), state("c

leaning", 1)], ("machine2", 1) => Any[state("on", 2), state("idle", 0), state("cleaning", 3), state("off", 0), state("on", 7), state("idle", 0), state("cleaning", 3)])


In [331]:
test = Dict(("machine1", 1) => [state("off", -1), state("on", 3), state("idle", 0), state("cleaning", 3), state("off", -1), state("on", 7), state("temp", 0)])

println(find_machine_activity(test["machine1", 1], 2))
println(find_machine_state(test["machine1", 1], 2))
println(convert_to_tp_sequence(11, test["machine1", 1]))

on
(state("on", 3), 2)
Any["on", "on", "cleaning", "cleaning", "cleaning", "on", "on", "on", "on", "on", "on"]


In [332]:
𝓜, x, resource_volume, slack, dicts = build_model(𝓓, schedules_dict)
println()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-07



In [333]:
# These are normalised constraints it is easier to think of it like:
# resource_volume[1,silo1] == resource_volume[0,silo1] -5*machine1[1]
# current_time_volume = previous_time_volume + rate_of_change_at_current_time*relevant_schedules_binary_variable
# println(𝓜)

In [334]:
optimize!(𝓜)
println(termination_status(𝓜))

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 306 rows, 186 columns and 637 nonzeros
Model fingerprint: 0xcacebc1d
Variable types: 184 continuous, 2 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 306 rows and 186 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 4 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.000000000000e+00, best bound 4.000000000000e+00, gap 0.0000%

User-callback calls 304, time in user-callback 0.00 sec
OPTIMAL


In [335]:
headers = ["period", "silo1", "slack_silo1", "machine1", "silo2", "slack_silo2", "machine2", "silo3", 
                                "slack_silo3", "cleaner1", "slack_cleaner1"]

format_output(𝓓, 𝓜, resource_volume, slack, x, dicts, headers)

Row,period,silo1,slack_silo1,machine1,silo2,slack_silo2,machine2,silo3,slack_silo3,cleaner1,slack_cleaner1
,Int64,Float64,Any,Any,Float64,Any,Any,Float64,Any,Float64,Any
1,0,900.0,,,200.0,,,300.0,,1.0,
2,1,895.0,0.0,on,200.0,0.0,on,320.0,0.0,1.0,0.0
3,2,890.0,0.0,on,200.0,0.0,on,340.0,0.0,1.0,0.0
4,3,890.0,0.0,cleaning,200.0,0.0,cleaning,340.0,0.0,-1.0,1.0
5,4,890.0,0.0,cleaning,200.0,0.0,cleaning,340.0,0.0,-1.0,1.0
6,5,890.0,0.0,cleaning,200.0,0.0,cleaning,340.0,0.0,-1.0,1.0
7,6,890.0,0.0,off,190.0,0.0,on,360.0,0.0,1.0,0.0
8,7,890.0,0.0,off,180.0,0.0,on,380.0,0.0,1.0,0.0
9,8,885.0,0.0,on,180.0,0.0,on,400.0,0.0,1.0,0.0


In [336]:
dicts.num_schedules_start = deepcopy(dicts.num_schedules_end)
dicts.schedules, dicts.num_schedules_end = generate_schedules_using_next_decent(𝓓, dicts.schedules, dicts.num_schedules_end, slack, x)
𝓜, dicts = update_model(𝓓, 𝓜, x, dicts)
optimize!(𝓜)

7
7


Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 306 rows, 192 columns and 717 nonzeros
Model fingerprint: 0x9f2876da
Variable types: 184 continuous, 8 integer (8 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+03]

MIP start from previous solve produced solution with objective 4 (0.00s)
Loaded MIP start from previous solve with objective 4

Presolve removed 291 rows and 180 columns
Presolve time: 0.01s
Presolved: 15 rows, 12 columns, 40 nonzeros
Variable types: 6 continuous, 6 integer (5 binary)

Root relaxation: objective 0.000000e+00, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl | 

In [337]:
# investigating the .lp files for both methods (bug or workaround) the model seems to be almost identical
# the only difference I can notice is the ordering 
# write_to_file(𝓜, "bugFinder.lp")

In [338]:
optimize!(𝓜)
println(termination_status(𝓜))

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 306 rows, 192 columns and 717 nonzeros
Model fingerprint: 0x9f2876da
Variable types: 184 continuous, 8 integer (8 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+03]
Presolved: 15 rows, 12 columns, 40 nonzeros

Continuing optimization...


Explored 1 nodes (3 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 2: 0 4 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%

User-callback calls 27, time in user-callback 0.00 sec
OPTIMAL


In [339]:
# These constraint are what we expect to see (no matter whether we choose buggy method or not)
# println(𝓜)

In [340]:
headers = ["period", "silo1", "slack_silo1", "machine1", "silo2", "slack_silo2", "machine2", "silo3", 
                                "slack_silo3", "cleaner1", "slack_cleaner1"]

format_output(𝓓, 𝓜, resource_volume, slack, x, dicts, headers)

Row,period,silo1,slack_silo1,machine1,silo2,slack_silo2,machine2,silo3,slack_silo3,cleaner1,slack_cleaner1
,Int64,Float64,Any,Any,Float64,Any,Any,Float64,Any,Float64,Any
1,0,900.0,,,200.0,,,300.0,,1.0,
2,1,895.0,0.0,on,200.0,0.0,on,320.0,0.0,1.0,0.0
3,2,890.0,0.0,on,200.0,0.0,on,340.0,0.0,1.0,0.0
4,3,890.0,0.0,cleaning,200.0,0.0,idle,340.0,0.0,0.0,0.0
5,4,890.0,0.0,cleaning,200.0,0.0,idle,340.0,0.0,0.0,0.0
6,5,890.0,0.0,cleaning,200.0,0.0,idle,340.0,0.0,0.0,0.0
7,6,890.0,0.0,off,200.0,0.0,cleaning,340.0,0.0,0.0,0.0
8,7,890.0,0.0,off,200.0,0.0,cleaning,340.0,0.0,0.0,0.0
9,8,890.0,0.0,off,200.0,0.0,cleaning,340.0,0.0,0.0,0.0
